In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('../input/biases-dataset/ibc_data.csv')

In [ ]:
df.head()

In [4]:
import re

# clean text from noise
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

df['text'] = df.text.apply(clean_text)

In [5]:
from transformers import AutoTokenizer

SEQ_LEN = 350

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=SEQ_LEN,
                                  truncation=True, padding='max_length',
                                  add_special_tokens=True, return_attention_mask=True,
                                  return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

# initializing two array for imput tensors
Xids = np.zeros((len(df), SEQ_LEN))
Xmask = np.zeros((len(df), SEQ_LEN))

for i, sentence in enumerate(df['text']):
    Xids[i, :], Xmask[i, :] = tokenize(sentence)
    if i % 1000 == 0:
        print(i)  # do this so we can see some progress

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

0
1000
2000
3000
4000


In [6]:
from sklearn.preprocessing import LabelEncoder
import pickle
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

pickle_out = open("LabelEncoder.pickle","wb")
pickle.dump(le, pickle_out)
pickle_out.close()

In [7]:
arr = df['label'].values  # take sentiment column in df as array
labels = np.zeros((arr.size, arr.max()+1))  # initialize empty (all zero) label array
labels[np.arange(arr.size), arr] = 1  # add ones in indices where we have a value

In [8]:

with open('biases-xids.npy', 'wb') as f:
    np.save(f, Xids)
with open('biases-xmask.npy', 'wb') as f:
    np.save(f, Xmask)
with open('biases-labels.npy', 'wb') as f:
    np.save(f, labels)

In [9]:
import tensorflow as tf

BATCH_SIZE = 32

# load arrays into tensorflow dataset
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

# create a mapping function that we use to restructure our dataset
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# using map method to apply map_func to dataset
dataset = dataset.map(map_func)
# shuffle data and batch it
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

In [10]:
# get the length of the batched dataset
DS_LEN = len([0 for batch in dataset])
SPLIT = 0.8  # 80-20 split

train = dataset.take(round(DS_LEN*SPLIT))  # get first 90% of batches
val = dataset.skip(round(DS_LEN*SPLIT))  # skip first 90% and keep final 10%

del dataset  # optionally, delete dataset to free up disk-space

## Model Definition

In [11]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(350,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(350,), name='attention_mask', dtype='int32')

# we consume the last_hidden_state tensor from bert (discarding pooled_outputs)
embeddings = bert(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.LSTM(64)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(64, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(X)

# define input and output layers of our model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze the BERT layer - otherwise we will be training 100M+ parameters...
model.layers[2].trainable = False

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
loss = tf.keras.losses.CategoricalCrossentropy()  # categorical = one-hot
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
checkpoint = tf.keras.callbacks.ModelCheckpoint("./Biases_Checkpoint",
                             monitor="accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)
earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', # value being monitored for improvement
                          min_delta = 0, #Abs value and is the min change required before we stop
                          patience = 5, #Number of epochs we wait before stopping 
                          verbose = 1,
                          restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'accuracy', factor = 0.01, patience = 3,
                                                 verbose = 1, min_delta = 0.0001)

callbacks = [checkpoint, earlystop, reduce_lr]

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = model.fit(train,
                    validation_data=val,
                    epochs=50,
                    callbacks = callbacks)

Epoch 1/50
109/109 [==============================] - 88s 647ms/step - loss: 1.1055 - accuracy: 0.4398 - val_loss: 0.9556 - val_accuracy: 0.5477

Epoch 00001: accuracy improved from -inf to 0.46359, saving model to ./Biases_Checkpoint


In [ ]:
history_dict = history.history
accuracy = max(history_dict['accuracy'])
val_acc = 0.9893

In [ ]:
import pickle 

pickle_out = open("BIASES_history_{}_{}.pickle".format(accuracy, val_acc),"wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

In [ ]:
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
model.save("./accuracy-{}-{}".format(accuracy, val_acc))

In [ ]:
import shutil
# shutil.make_archive("model", 'zip', './accuracy-0.9601491093635559-0.9893')

In [ ]:
text = clean_text('But the most important step that Democrats can take is to transfer the fight for their agenda from the realm of pressure groups , where they are vulnerable , to the electorate , where they hold the advantage .')
print(model.predict(text))